# Getting Structured Outputs from Llama3 using Ollama 

In [ ]:
!pip install -U ollama pydantic

In [6]:
from ollama import chat
from pydantic import BaseModel
from typing import Optional, List

class Task(BaseModel):
    title: str
    description: str
    priority: str  # "High", "Medium", "Low"
    estimated_hours: float
    dependencies: List[str] = []
    status: str = "Not Started"  # "Not Started", "In Progress", "Completed"
    assigned_to: Optional[str] = None

response = chat(
    messages=[
        {
            'role': 'user',
            'content': 'Create a task for implementing a new feature in our project management software that allows users to track time spent on tasks. Include dependencies and make it realistic.',
        }
    ],
    model='llama3.1',
    format=Task.model_json_schema(),
)

task = Task.model_validate_json(response['message']['content'])
print(f"Task: {task.title}")
print(f"Priority: {task.priority}")
print(f"Estimated Hours: {task.estimated_hours}")
print(f"Dependencies: {', '.join(task.dependencies)}")
print(f"Status: {task.status}")

Task: Implement Time Tracking Feature
Priority: High
Estimated Hours: 120.0
Dependencies: 
Status: Not Started


We can turn this structured output functionality into a function calling pipeline.